<a href="https://colab.research.google.com/github/Gituhin/Sentence-Classification-naive-bayes-/blob/main/Sentence_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing necessary libraries

In [ ]:
from google.colab import auth
auth.authenticate_user()
import string 
import gspread
from oauth2client.client import GoogleCredentials
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pandas as pd

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


loading the train dataset

In [ ]:
gc = gspread.authorize(GoogleCredentials.get_application_default())
worksheet = gc.open('traindata').sheet1
rows = worksheet.get_all_values()
# Convert to a DataFrame and render.
data=pd.DataFrame.from_records(rows)

Line preprocessing function

In [ ]:
def preprocessline(data, c):
  step=str.maketrans('','',string.punctuation)    #removing punctuations
  line=data[1][c].translate(step) 
  line=line.lower()       #converting all words to lower cases and splitting them
  words=line.split()

  st_words=stopwords.words('english')
  i=0
  while i<len(words):
    if words[i] in st_words:     #removing the stop words, finding them and poping them out from the words splitted list
      words.pop(i)
      i=0
    i+=1
  return words

Storing unique words in each category in corresponding sets

In [ ]:
word_set=set()
for c in range(1, len(data)):           #unique collections of words in a set to avoid repetitions
  word_set.update(preprocessline(data, c))

In [ ]:
science=set()                 
covid=set()                       #set of words in each category which would be used as reference for creating vocabulary
business=set()
sports=set()
st_words=stopwords.words('english')

for labels in range(1, len(data)):
  if data[0][labels]=='science':
    words=preprocessline(data, labels)
    science.update(words)

  
  elif data[0][labels]=='covid':
    words=preprocessline(data, labels)
    covid.update(words) 
  
  elif data[0][labels]=='business':
    words=preprocessline(data, labels)
    business.update(words)
  
  else:
    words=preprocessline(data, labels)
    sports.update(words)


Creating Vocabulary with frequencies of words in a class.
*classes are viz:* 

*   science
*   covid

*   business
*   sports








In [ ]:
words_in_science={}
words_in_covid={}
words_in_business={}          #dictionaries used for each category, the key stores the words and the corresponding values store the frequency
words_in_sports={}

for p in science:
  words_in_science[p]=0
for p in business:
  words_in_business[p]=0        #initially setting frequencies of all words=0
for p in covid:
  words_in_covid[p]=0
for p in sports:
  words_in_sports[p]=0


for labels in range(1, len(data)):
  if data[0][labels]=='science':
    words=preprocessline(data, labels)          #iterating the frequencies of the words in different categories
    for wd in words:
      words_in_science[wd]+=1

  elif data[0][labels]=='business':
    words=preprocessline(data, labels)
    for wd in words:
      words_in_business[wd]+=1


  elif data[0][labels]=='covid':
    words=preprocessline(data, labels)
    for wd in words:
      words_in_covid[wd]+=1

  else:
    words=preprocessline(data, labels)
    for wd in words:
      words_in_sports[wd]+=1


Calculating Prior distributions of classes
[no. of times the class occurs/total no. of classes]

In [ ]:
class_labels=set()
for label in range(1, len(data)):
  class_labels.add(data[0][label])
category={}

for label in class_labels:
  freq=0
  for i in range(1, len(data)):
   if label==data[0][i]:
    freq+=1
  category[label]=freq

print('the prior distributions of classes are:')
p_business=category['business']/(len(data)-1)
p_science=category['science']/(len(data)-1)
p_sports=category['sports']/(len(data)-1)
p_covid=category['covid']/(len(data)-1)

print("business=",p_business)
print("covid=", p_covid )
print("science=", p_science )
print("sports=", p_sports )

the prior distributions of classes are:
business= 0.2375
covid= 0.2625
science= 0.25
sports= 0.25


# Class conditional probability of each word in document


---

no. of time the word occurs in a particular class/total no. of words in that class


In [ ]:
total_science, total_covid, total_business, total_sports=0,0,0,0

for t in words_in_business:
  total_business+=words_in_business[t]
for t in words_in_science:                      #summing up the total number of words in each categories for total no. of words in a class
  total_science+=words_in_science[t]
for t in words_in_covid:
  total_covid+=words_in_covid[t]
for t in words_in_sports:
  total_sports+=words_in_sports[t]

prob_science={}
prob_covid={}
prob_business={}          #dictionaries used for storing the class conditional probailities of words
prob_sports={}

for a in words_in_business:
  prob_business[a]=words_in_business[a]/total_business
for a in words_in_science:
  prob_science[a]=words_in_science[a]/total_science
for a in words_in_covid:
  prob_covid[a]=words_in_covid[a]/total_covid
for a in words_in_sports:
  prob_sports[a]=words_in_sports[a]/total_sports

for wdr in word_set:
  if wdr not in science:
    prob_science[wdr]=0.00001
  if wdr not in sports:                   #if the words is not in the sentence then assigning it a small probability almost near 0, but should not be 0
    prob_sports[wdr]=0.00001
  if wdr not in covid:
    prob_covid[wdr]=0.00001
  if wdr not in business:
    prob_business[wdr]=0.00001


Applying the trained model in test dataset

In [ ]:
worksheet1= gc.open('testdata').sheet1
rows = worksheet1.get_all_values()
test_data=pd.DataFrame.from_records(rows)

# Calculating posterier distribution of given sentences over classes

---

by Naive Bayes formula we calculate the posterior distribution of each sentence
belonging to a particular class.

In [ ]:
for c in range(1, len(test_data)):
  words1=preprocessline(test_data, c)

  for wds in words1:
    if wds not in word_set:
      prob_business[wds]=0.00001
      prob_science[wds]=0.00001
      prob_covid[wds]=0.00001                   #if the word in testset not in trainset then assigning a small probabilty 
      prob_sports[wds]=0.00001

  postp_science=1
  postp_business=1
  postp_covid=1
  postp_sports=1

  for wds in words1:
    postp_science*=prob_science[wds]
    postp_business*=prob_business[wds]
    postp_covid*=prob_covid[wds]
    postp_sports*=prob_sports[wds]

  postp_science=postp_science*p_science/(1.00e-20)
  postp_covid=postp_covid*p_covid/(1.00e-20)                          #here the denominator/the probabilty of given sentence is same for all hence 1.00e-20 has been
  postp_business=postp_business*p_business/(1.00e-20)             #choosen as an appropiate upscaling factor since all are affected by same factor.
  postp_sports=postp_sports*p_sports/(1.00e-20)
  print('the probability of sentence %d being science is '%c, postp_science)
  print('the probability of sentence %d being covid is '%c, postp_covid)
  print('the probability of sentence %d being business is '%c, postp_business)
  print('the probability of sentence %d being sports is '%c, postp_sports)
  print('\n')
    

I have winded up till posterior distribution, next we can select the class having the max. probabilty. All the predicted results do match the test categories.